In [ ]:
import sys
import os
import yaml
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import sklearn.preprocessing
import sklearn.cluster

import pandas as pd
pd.options.mode.chained_assignment = None

sys.path.append('../python/')
from analysis_util import separate_concentrations_into_cohorts_and_clean, add_concentration_diffs_to_one_cohort_table
from dashboard_notebook_util import read_clean_adelaide_randomisation_schedule

# Plotly Dash doesn't display error messages well, so we use logging 
import logging
logging.basicConfig(level=logging.DEBUG, filename="dash_logs.log")

LIST_COHORT_NAMES = ["Melbourne", "Adelaide"]

In [ ]:
dict_parameters = yaml.safe_load(open("../parameters/july_2024_data_parameters.yaml", "r"))

In [ ]:
pd_df_estimated_concentrations = pd.read_csv(
    open(
        os.path.join(
            dict_parameters["base directory path"],
            dict_parameters["output directory"],
            dict_parameters["estimated concentrations file name"]
        ),
        "rb"
    )
)

In [ ]:
pd_df_estimated_concentrations

In [ ]:
pd_df_adelaide_randomisation_schedule = read_clean_adelaide_randomisation_schedule(dict_parameters)

In [ ]:
pd_df_adelaide_randomisation_schedule

In [ ]:
dict_pd_df_cohort_tables = separate_concentrations_into_cohorts_and_clean(
    dict_parameters,
    pd_df_estimated_concentrations
)

In [ ]:
#TODO: remove the extra index column
pd_df_adelaide = dict_pd_df_cohort_tables["Adelaide"].copy()

In [ ]:
pd_df_adelaide

In [ ]:
pd_df_adelaide = pd_df_adelaide.merge(
    pd_df_adelaide_randomisation_schedule,
    on="patient number",
    how="left"
)

In [ ]:
pd_df_adelaide

In [ ]:
pd_df_adelaide["day"] = 1
pd_df_adelaide["patient number"] = pd_df_adelaide["patient number"].astype(str)
pd_df_adelaide.loc[pd_df_adelaide["patient number"].str.endswith("D2"), "day"] = 2
pd_df_adelaide

In [ ]:
pd_df_adelaide["patient number"] = (
    pd_df_adelaide["patient number"].str.replace('-D2', '')
)

In [ ]:
pd_df_adelaide_is_treated = (
    pd_df_adelaide[pd_df_adelaide["treatment"] == 1].set_index(["patient number", "time code", "time int"], drop=True)
)
pd_df_adelaide_not_treated = (
    pd_df_adelaide[pd_df_adelaide["treatment"] == 0].set_index(["patient number", "time code", "time int"], drop=True)
)
pd_df_adelaide_treatment_differences = (pd_df_adelaide_is_treated - pd_df_adelaide_not_treated).reset_index()
add_concentration_diffs_to_one_cohort_table(dict_parameters, pd_df_adelaide_treatment_differences, 1)

In [ ]:
pd_df_adelaide_pre_differenced = pd_df_adelaide.copy()
add_concentration_diffs_to_one_cohort_table(dict_parameters, pd_df_adelaide_pre_differenced, 1, use_day = True)

In [ ]:
pd_df_adelaide_pre_differenced

In [ ]:
pd_df_adelaide_pre_differenced_is_treated = (
    pd_df_adelaide_pre_differenced[pd_df_adelaide_pre_differenced["treatment"] == 1].set_index(
        ["patient number", "time code", "time int"], drop=True
    )
)
pd_df_adelaide_pre_differenced_not_treated = (
    pd_df_adelaide_pre_differenced[pd_df_adelaide_pre_differenced["treatment"] == 0].set_index(
        ["patient number", "time code", "time int"], drop=True
    )
)
pd_df_adelaide_pre_differenced_differences = (
    pd_df_adelaide_pre_differenced_is_treated - pd_df_adelaide_pre_differenced_not_treated
).reset_index()

In [ ]:
pd_df_adelaide_pre_differenced_differences

Do some clustering on the Melbourne patients

In [ ]:
list_pivot_values_columns = [
    str_column_name for str_column_name in dict_pd_df_cohort_tables["Melbourne"].columns
    if str_column_name not in list_first_columns + ["time int"]
]
pd_df_melbourne_concentrations_pivoted = (
    dict_pd_df_cohort_tables["Melbourne"].pivot(
        index="patient number", 
        columns="time code", 
        values=list_pivot_values_columns
    )
)
pd_df_melbourne_concentrations_pivoted.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col for col in pd_df_melbourne_concentrations_pivoted.columns.values
]
pd_df_melbourne_concentrations_pivoted = pd_df_melbourne_concentrations_pivoted.reset_index()

In [ ]:
pd_df_melbourne_concentrations_pivoted

In [ ]:
list_time_codes_for_clustering = ['C', 'D', 'E']
list_columns_to_cluster = []
for str_time_code in list_time_codes_for_clustering:
    list_columns_to_cluster += [
        "IP-10 diff_" + str_time_code, 
        "TNF-a diff_" + str_time_code, 
        "IFN-gamma diff_" + str_time_code, 
        "IL-6 diff_" + str_time_code, 
    ]

scaler_object = sklearn.preprocessing.StandardScaler()
np_arr_scaled_data = scaler_object.fit_transform(pd_df_melbourne_concentrations_pivoted[list_columns_to_cluster])

num_clusters = 5
kmeans_object = sklearn.cluster.KMeans(n_clusters=num_clusters, random_state=42)
np_arr_cluster_labels = kmeans_object.fit_predict(np_arr_scaled_data)

pd_df_melbourne_concentrations_pivoted['cluster'] = np_arr_cluster_labels

In [ ]:
dict_pd_df_cohort_tables["Melbourne"] = dict_pd_df_cohort_tables["Melbourne"].merge(
    pd_df_melbourne_concentrations_pivoted[["patient number", "cluster"]],
    on = "patient number",
    how = "left"
)
dict_pd_df_cohort_tables["Adelaide"]["cluster"] = 1
dict_pd_df_cohort_tables["Melbourne"]["cluster"] = 1

In [ ]:
dict_pd_df_cohort_tables["Melbourne"]

In [ ]:
dash_app_object_boxes = dash.Dash(__name__)

# Define the layout
dash_app_object_boxes.layout = html.Div([
    html.H1("Plots of estimated concentrations"),

    html.Div([
        html.Label("Cohort:"),
        dcc.Dropdown(
            id='cohort-dropdown',
            options=[{'label': col, 'value': col} for col in LIST_COHORT_NAMES],
            value=LIST_COHORT_NAMES[0]
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),
    

    html.Div([
        html.Label("Plot type:"),
        dcc.Dropdown(
            id='plot-type-dropdown',
            options=[
                {'label': 'box', 'value': 'box'},
                {'label': 'strip', 'value': 'strip'},
            ],
            value='box',
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Data type:"),
        dcc.Dropdown(
            id='data-type-dropdown',
            options=[
                {'label': 'raw estimates', 'value': 'raw estimates'},
                {'label': 'differences', 'value': 'differences'},
            ],
            value='raw estimates',
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Analyte:"),
        dcc.Dropdown(
            id='analyte-dropdown',
            options=[{'label': col, 'value': col} for col in dict_parameters["list of analytes"]],
            value=dict_parameters["list of analytes"][0]
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),
    
    dcc.Graph(id='scatter-plot')
], style={'backgroundColor': 'white', 'padding': '20px'})


@dash_app_object_boxes.callback(
    Output('scatter-plot', 'figure'),
    Input('cohort-dropdown', 'value'),
    Input('analyte-dropdown', 'value'),
    Input('data-type-dropdown', 'value'),
    Input('plot-type-dropdown', 'value'),
)
def update_graph(str_cohort_name, str_analyte, str_data_type, str_plot_type):
    
    if str_data_type == "raw estimates":
        str_column_name = dict_parameters["column name prefix for estimated concentrations"] + str_analyte
    elif str_data_type == "differences":
        str_column_name = str_analyte + " diff"
    
    if str_plot_type == "box":
        fig = px.box(
            dict_pd_df_cohort_tables[str_cohort_name], 
            x = "time code", 
            y = str_column_name,
            color="cluster",
        )
    elif str_plot_type == "strip":
        fig = px.strip(
            dict_pd_df_cohort_tables[str_cohort_name], 
            x = "time code", 
            y = str_column_name,
            hover_name = "patient number",
            color="cluster",
        )    
        fig.update_traces(marker = dict(opacity = 0.75))
    
    if str_data_type == "raw estimates":
        str_y_label = "concentration"
    elif str_data_type == "differences":
        str_y_label = "diff. in concentration"
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis_title="time",
        # xaxis=dict(
        #     tickmode='array',
        #     tickvals=['A', 'B', 'C', 'D', 'E'],
        #     ticktext=['minus 1wk', 'minus 1hr', 'plus 3hr', 'plus 7hr', 'plus 25hr'],
        # ),
        yaxis_title=str_y_label,
        font=dict(
            family="Arial",
            size=16,
            color="black"
        )
    )
    return fig

# Run the app
if __name__ == '__main__':
    dash_app_object_boxes.run(jupyter_mode="inline", debug = True, port=3312)

In [ ]:
dash_app_object_lines = dash.Dash(__name__)

list_patient_ids_melbourne = list(dict_pd_df_cohort_tables["Melbourne"]["patient number"].unique())
list_patient_ids_adelaide = list(dict_pd_df_cohort_tables["Adelaide"]["patient number"].unique())
list_patient_id_dropdown_dicts = [
    {'label': str_patient_id, 'value': str_patient_id} for str_patient_id in list_patient_ids_adelaide + list_patient_ids_melbourne
]

# Define the layout
dash_app_object_lines.layout = html.Div([
    html.H1("Line plots by patient number"),

    html.Div([
        html.Label("Patient number:"),
        dcc.Dropdown(
            id='patient-number-dropdown',
            options=list_patient_id_dropdown_dicts,
            value=list_patient_id_dropdown_dicts[0]["value"],
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Data type:"),
        dcc.Dropdown(
            id='data-type-dropdown',
            options=[
                {'label': 'raw estimates', 'value': 'raw estimates'},
                {'label': 'normalised raw estimates', 'value': 'normalised raw estimates'},
                {'label': 'differences', 'value': 'differences'},
            ],
            value='raw estimates',
        ),
    ], style={'width': '20%', 'display': 'inline-block'}),

    dcc.Graph(id='line-plot')
], style={'backgroundColor': 'white', 'padding': '20px'})


@dash_app_object_lines.callback(
    Output('line-plot', 'figure'),
    Input('patient-number-dropdown', 'value'),
    Input('data-type-dropdown', 'value'),
)
def update_graph(str_patient_number, str_data_type):
    
    fig = go.Figure()
    if str_patient_number in list_patient_ids_melbourne:
        pd_df_patients_data = (
            dict_pd_df_cohort_tables["Melbourne"][dict_pd_df_cohort_tables["Melbourne"]["patient number"] == str_patient_number]
        )
    elif str_patient_number in list_patient_ids_adelaide:
        pd_df_patients_data = (
            dict_pd_df_cohort_tables["Adelaide"][dict_pd_df_cohort_tables["Adelaide"]["patient number"] == str_patient_number]
        )

    for str_analyte in ["IP-10", "IFN-gamma", "IL-6", "TNF-a"]:
        
        if str_data_type in ["raw estimates", "normalised raw estimates"]:
            str_column_name = dict_parameters["column name prefix for estimated concentrations"] + str_analyte
        elif str_data_type == "differences":
            str_column_name = str_analyte + " diff"
        pd_series_y = pd_df_patients_data[str_column_name]
        if str_data_type == "normalised raw estimates":
            pd_series_y = (pd_series_y - pd_series_y.mean()) / (pd_series_y.max() - pd_series_y.min())
        fig.add_trace(go.Scatter(x=pd_df_patients_data["time int"], y=pd_series_y, mode='lines', name=str_analyte))
    
    if str_data_type == "raw estimates":
        str_y_label = "concentration (pg/ml)"
    if str_data_type == "normalised raw estimates":
        str_y_label = "normalised concentration"
    elif str_data_type == "differences":
        str_y_label = "diff. in concentration (pg/ml)"
        
    if str_patient_number in list_patient_ids_melbourne:
        dict_x_ticks = dict(
            tickmode='array',
            tickvals=[1, 2, 3, 4, 5],
            ticktext=['minus 1wk', 'minus 1hr', 'plus 3hr', 'plus 7hr', 'plus 25hr'],
        )
    elif str_patient_number in list_patient_ids_adelaide:
        dict_x_ticks = dict(
            tickmode='array',
            tickvals=[1, 2, 3, 4, 5, 6, 7],
            ticktext=['minus 1hr', 'plus 15min', 'plus 30min', 'plus 1hr', 'plus 2hr', 'plus 4hr', 'plus 8hr'],
        )
        
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis_title="time",
        # title={
        #      'text': f'Concentrations for Patient {str_patient_number}',
        #      'y':0.95,
        #      'x':0.5,
        #      'xanchor': 'center',
        #      'yanchor': 'top',
        #      'font': {'size': 24}
        # },
        xaxis=dict_x_ticks,
        yaxis_title=str_y_label,
        font=dict(
            family="Arial",
            size=16,
            color="black"
        )
    )
    return fig

# Run the app
if __name__ == '__main__':
    dash_app_object_lines.run(jupyter_mode="inline", debug = True, port=3311)